## Twitter Sentiment Analisys menggunakan metode A NAIVE BAYES CLASSIFIER (NBC)
### 1.Case Folding:
Case folding dilakukan untuk
mengubah setiap karakter didalam teks menjadi huruf
kecil. Tidak semua kata dalam teks konsisten dalam
penggunaan huruf kapital disinilah tujuan dilakukan case
folding untuk mengkonversi setiap karakter dalam kata
menjadi huruf kecil.
### 2.Tokenisasi: 
Tokenisasi merupakan proses
pemecahan kata pada suatu teks ke dalam satuan kata.
Tokenisasi dilakukan untuk menghasilkan kumpulan kata
yang berdiri sendiri, tokenisasi memecah teks yang
semula berupa kalimat menjadi kata-kata. tokenisasi
menghilangkan delimeter seperti titik (.), koma (,), spasi,
dan karakter angka yang ada pada kata tersebut.
Dalam penelitian ini tokenisasi dilakukan untuk
memecah kata, serta melakukan penghapusan delimeter
beserta karakter angka bersama tweet entity seperti
hashtag, retweet dan mention.
### 3.Filtering: 
Filtering merupakan proses dalam text
preprocessing setelah tokenisasi, filtering dilakukan untuk
untuk mengambil kata penting hasil tokenisasi. Pada tahap
filtering kata akan ditentukan apakah akan digunakan atau 
dibuang. Proses dalam filtering dalam membuang katakata yang tidak digunakan atau stopword terdapat dalam
bag of words stoplist.
Stopword merupakan daftar kata-kata yang tidak
mempresentasikan isi dari suatu dokumen teks, stopword
dilakukan untuk meghilangkan kata atau term yang tidak
memiliki arti. Daftar stoplist akan dibuat sebelum
melakukan proses stopword removal, jika kata-kata
terdapat dalam daftar stoplist, maka kata tersebut akan
dihapus, sehingga kata-kata yang tersisa akan dianggap
kata yang mencirikan isi suatu dokumen.
### 4.Stemming: 
Stemming merupakan proses mengubah
kata menjadi bentuk dasarnya. Stemming dilakukan untuk
meyeragamkan bentuk kata. Tujuan dari proses stemming
adalah menghilangkan imbuhan-imbuhan baik itu berupa
prefiks, sufiks, maupun konfiks yang ada pada setiap kata. 
Stemming dalam penelitian ini dilakukan berdasarkan
aturan morfologi bahasa Indonesia.


### berikut adalah langka-langkahnya:

### 1.Install Library(module) yang dibutuhkan

In [ ]:
# %pip install tweepy
# %pip install Sastrawi
# %pip install numpy
# %pip install sklearn
# %pip install textblob
# %pip install pandas
# %pip install matplotlib

In [1]:
import tweepy

BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAJQrYQEAAAAA1BtlexYIrHRZvqkz6TEOraEC%2B0k%3D5cOp6U7ytEMpcTDctDQWVl6HFd0Y28pxhDnTGyHA9KiiC1MVCF'

client = tweepy.Client(bearer_token=BEARERTOKEN)

query = '"Ruang Guru" lang:id'

# get data max 100
responses = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at'])
tweet100 = [tweet for tweet in responses.data]

# get max 1000
tweettexts = [tweet.text for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=1000)]

In [29]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re

In [3]:
# tweettexts = [tweet.lower() for tweet in tweettexts]
from datetime import datetime

data = {
    'text' : tweettexts
}
df = pd.DataFrame(data)
df.to_csv(f"crawling{datetime.today().strftime('%Y-%m-%d')}.csv", mode='w', index=False, header=data.keys())

In [4]:
import pandas as pd

df = pd.read_csv('crawling2022-06-03.csv', encoding='unicode_escape')
df.head(5)

,text
0,RT @Muhisammy: @Androngehe @worksfess Sama kek...
1,"RT @RamenTerbang: @worksfess jangankan nakes, ..."
2,"@gapyearfess Ruang guru , zenius sama bimbel a..."
3,@jinniesteacher Kebalik kak..maksud dia. Murid...
4,"RT @RamenTerbang: @worksfess jangankan nakes, ..."


In [5]:
#Cleaning Text
def textcleaner(text):
    new_text = str(text)
    
    # remove old style retweet text "RT"
    new_text = re.sub(r'^RT\s+', '', new_text)
    
    # remove username
    new_text = re.sub(r'@([A-Za-z0-9_]+)', '', new_text)

    # remove hyperlinks
    new_text = re.sub(r'https?:\/\/.*[\r\n]*', '', new_text)

    # remove invalid character
    new_text = re.sub("[^A-Za-z" "]+", " ", new_text)
    
    new_text    = re.sub(r'\b\w(1,2)\b',' ', new_text) #menghilangkan 2 kata
    
    return new_text

In [11]:
def caselower(sentences):
    return sentences.lower()

In [6]:
def tokenmaker(sentences):
    return sentences.strip().split()

In [17]:
#Stop Removal
stopword = nltk.corpus.stopwords.words("indonesian")
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [36]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

In [23]:
# cleaning text
df['clean_text'] = df['text'].apply(lambda x : textcleaner(x))

In [24]:
# cover ke lower case
df['lower'] = df['clean_text'].apply(lambda x : caselower(x))

In [25]:
# tokenizing
df['token'] = df['lower'].apply(lambda x : tokenmaker(x))

In [26]:
# stopword removing
df["stop_removed"] = df['token'].apply(lambda x: remove_stopwords(x))

In [37]:
# stemming removing
df["stemmed"] = df['stop_removed'].apply(lambda x: stemming(x))
print("Stemming selesai!")

Stemming selesai!


In [38]:
df.head(df.size)

,text,clean_text,token,lower,stop_removed,stemmed
0,RT @Muhisammy: @Androngehe @worksfess Sama kek...,Sama kek guru itu julidin murid termasuk cara...,"[sama, kek, guru, itu, julidin, murid, termasu...",sama kek guru itu julidin murid termasuk cara...,"[kek, guru, julidin, murid, penyelesaiannya, r...","[kek, guru, julidin, murid, selesai, ruang, gu..."
1,"RT @RamenTerbang: @worksfess jangankan nakes, ...",jangankan nakes murid julidin gurunya aja uda...,"[jangankan, nakes, murid, julidin, gurunya, aj...",jangankan nakes murid julidin gurunya aja uda...,"[nakes, murid, julidin, gurunya, aja, udah, yg...","[nakes, murid, julidin, guru, aja, udah, yg, j..."
2,"@gapyearfess Ruang guru , zenius sama bimbel a...",Ruang guru zenius sama bimbel area eksak,"[ruang, guru, zenius, sama, bimbel, area, eksak]",ruang guru zenius sama bimbel area eksak,"[ruang, guru, zenius, bimbel, area, eksak]","[ruang, guru, zenius, bimbel, area, eksak]"
3,@jinniesteacher Kebalik kak..maksud dia. Murid...,Kebalik kak maksud dia Murid julidin guru Cum...,"[kebalik, kak, maksud, dia, murid, julidin, gu...",kebalik kak maksud dia murid julidin guru cum...,"[kebalik, kak, maksud, murid, julidin, guru, c...","[balik, kak, maksud, murid, julidin, guru, cum..."
4,"RT @RamenTerbang: @worksfess jangankan nakes, ...",jangankan nakes murid julidin gurunya aja uda...,"[jangankan, nakes, murid, julidin, gurunya, aj...",jangankan nakes murid julidin gurunya aja uda...,"[nakes, murid, julidin, gurunya, aja, udah, yg...","[nakes, murid, julidin, guru, aja, udah, yg, j..."
...,...,...,...,...,...,...
995,RT @middleclassthud: suka banget ama lagunya l...,suka banget ama lagunya letto yang ruang guru,"[suka, banget, ama, lagunya, letto, yang, ruan...",suka banget ama lagunya letto yang ruang guru,"[suka, banget, ama, lagunya, letto, ruang, guru]","[suka, banget, ama, lagu, letto, ruang, guru]"
996,RT @donhae: @AREAJULID kalo di negeriâ¦ awikw...,kalo di negeri awikwok banget dah isinya kerj...,"[kalo, di, negeri, awikwok, banget, dah, isiny...",kalo di negeri awikwok banget dah isinya kerj...,"[kalo, negeri, awikwok, banget, dah, isinya, k...","[kalo, negeri, awikwok, banget, dah, isi, kerj..."
997,RT @middleclassthud: suka banget ama lagunya l...,suka banget ama lagunya letto yang ruang guru,"[suka, banget, ama, lagunya, letto, yang, ruan...",suka banget ama lagunya letto yang ruang guru,"[suka, banget, ama, lagunya, letto, ruang, guru]","[suka, banget, ama, lagu, letto, ruang, guru]"
998,@moonareas ruang guruð­ð­ð­ð­ð­,ruang guru,"[ruang, guru]",ruang guru,"[ruang, guru]","[ruang, guru]"


In [ ]:
pos_kata = open('kata_positif_2.txt', 'r').readlines()
neg_kata = open('kata_negatif_2.txt', 'r').readlines()

def labeling(tokens):
    
    count_p = 0 #nilai positif
    count_n = 0 #nilai negatif
    count_ne = 0 #nilai netral
    
    for token in tokens:
        for kata_pos in pos_kata:
            if kata_pos.strip().lower() == token.lower():
                #ind-1 digunakan untuk mencari nilai index sebelum index positifnya
                #aku menyukai bola
                if stemmedtexts[ind-1] in negasi:
                    print(stemmedtexts[ind-1], kata_pos, ['negatif'])
                    count_n += 1
                else:
                    print(kata_pos, ['positif'])
                    count_p += 1

        for kata_neg in neg_kata:
            if kata_neg.strip().lower() == tweet.lower():
                if stemmedtexts[ind-1] in negasi:
                    print(stemmedtexts[ind-1], kata_neg, ['positif'])
                    count_p += 1
                else:
                    print(kata_neg, ['negatif'])
                    count_n += 1
        
        for kat_pos_neg in (pos_kata + neg_kata):
            if kat_pos_neg.strip().lower() != tweet.lower():
                print(stemmedtexts[ind-1], kat_pos_neg, ['netral'])
                count_ne += 1
    
    sentiments.append()
    


In [9]:
pos_kata = open('kata_positif_2.txt', 'r').readlines()
neg_kata = open('kata_negatif_2.txt', 'r').readlines()

sentiments = [] #Membuat sebuah list menyimpan nilai sentiment

#list kata-kata negasi
negasi = open('negasi.txt', 'r').readlines()

#fungsi menghitung sentiment setiap tweet
for ind,stemmed in enumerate(stemmedtexts):
    print(stemmed.strip())
    tweets = stemmed.strip().split() #tokenization
    # print(tweets)
    
    count_p = 0 #nilai positif
    count_n = 0 #nilai negatif
    count_ne = 0 #nilai netral
    
    for tweet in tweets:
        for kata_pos in pos_kata:
            if kata_pos.strip().lower() == tweet.lower():
                #ind-1 digunakan untuk mencari nilai index sebelum index positifnya
                #aku menyukai bola
                if stemmedtexts[ind-1] in negasi:
                    print(stemmedtexts[ind-1], kata_pos, ['negatif'])
                    count_n += 1
                else:
                    print(kata_pos, ['positif'])
                    count_p += 1

        for kata_neg in neg_kata:
            if kata_neg.strip().lower() == tweet.lower():
                if stemmedtexts[ind-1] in negasi:
                    print(stemmedtexts[ind-1], kata_neg, ['positif'])
                    count_p += 1
                else:
                    print(kata_neg, ['negatif'])
                    count_n += 1
        
        for kat_pos_neg in (pos_kata + neg_kata):
            if kat_pos_neg.strip().lower() != tweet.lower():
                print(stemmedtexts[ind-1], kat_pos_neg, ['netral'])
                count_ne += 1
    
    sentiments.append()

NameError: name 'stemmedtexts' is not defined